<a href="https://colab.research.google.com/github/scgupta/yearn2learn/blob/master/speech/asr/python_speech_recognition_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Speech Recognition with Python

There are several Automated Speech Recognition (ASR) alternatives, and most of them have bindings for Python. There are two kinds of solutions:

- **Service:** These run on the cloud, and are accessed either through REST endpoints or Python library. Examples are cloud speech services from Google, Amazon, Microsoft.
- **Software:** These run locally on the machine (not requiring network connection). Examples are CMU Sphinx and Mozilla DeepSpeech.

Speech Recognition APIs are of two types:
- **Batch:** The full audio file is passed as parameter, and speech-to-text transcribing is done in one shot.
- **Streaming:** The chunks of audio buffer are repeatedly passed on, and intermediate results are accessible.

All packages support batch mode, and some support streaming mode too.

One common use case is to collect audio from microphone and passes on the buffer to the speech recognition API. Invariably, in such transcribers, microphone is accessed though [PyAudio](https://people.csail.mit.edu/hubert/pyaudio/), which is implemented over [PortAudio](http://www.portaudio.com/).

From Colab menu, select: **Runtime** > **Change runtime type**, and verify that it is set to Python3, and select GPU if you want to try out GPU version.

## Common Setup

1. **Install google cloud speech package**

You may have to restart the runtime after this.

In [0]:
!pip3 install google-cloud-speech

     |████████████████████████████████| 92kB 3.4MB/s 


2. **Download audio files for testing**

Following files will be used as test cases for all speech recognition alternatives covered in this notebook.

In [0]:
!curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.6.0/audio-0.6.0.tar.gz
!tar -xvzf audio-0.6.0.tar.gz
!ls -l ./audio/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   608    0   608    0     0   2632      0 --:--:-- --:--:-- --:--:--  2620
100  192k  100  192k    0     0   310k      0 --:--:-- --:--:-- --:--:--  310k
audio/
audio/2830-3980-0043.wav
audio/Attribution.txt
audio/4507-16021-0012.wav
audio/8455-210777-0068.wav
audio/License.txt
total 260
-rw-r--r-- 1 501 staff 63244 Nov 18  2017 2830-3980-0043.wav
-rw-r--r-- 1 501 staff 87564 Nov 18  2017 4507-16021-0012.wav
-rw-r--r-- 1 501 staff 82924 Nov 18  2017 8455-210777-0068.wav
-rw-r--r-- 1 501 staff   340 May 14  2018 Attribution.txt
-rw-r--r-- 1 501 staff 18652 May 12  2018 License.txt


3. **Define test cases**

In [0]:
TESTCASES = [
  {
    'filename': 'audio/2830-3980-0043.wav',
    'text': 'experience proves this',
    'encoding': 'LINEAR16',
    'lang': 'en-US'
  },
  {
    'filename': 'audio/4507-16021-0012.wav',
    'text': 'why should one halt on the way',
    'encoding': 'LINEAR16',
    'lang': 'en-US'
  },
  {
    'filename': 'audio/8455-210777-0068.wav',
    'text': 'your power is sufficient i said',
    'encoding': 'LINEAR16',
    'lang': 'en-US'
  }
]

4. **Utility Functions**

In [0]:
from typing import Tuple
import wave

def read_wav_file(filename) -> Tuple[bytes, int]:
    with wave.open(filename, 'rb') as w:
        rate = w.getframerate()
        frames = w.getnframes()
        buffer = w.readframes(frames)

    return buffer, rate

def simulate_stream(buffer: bytes, batch_size: int = 4096):
    buffer_len = len(buffer)
    offset = 0
    while offset < buffer_len:
        end_offset = offset + batch_size
        buf = buffer[offset:end_offset]
        yield buf
        offset = end_offset


---

# Google

Google has [speech-to-text](https://cloud.google.com/speech-to-text/docs) as one of the Google Cloud services. It has [libraries](https://cloud.google.com/speech-to-text/docs/reference/libraries) in C#, Go, Java, JavaScript, PHP, Python, and Ruby. It supports both batch and stream modes.

## Setup

1. **Upload Google Cloud Cred file**

Have Google Cloud creds stored in a file named **`gc-creds.json`**, and upload it by running following code cell. See https://developers.google.com/accounts/docs/application-default-credentials for more details.

This may reqire enabling **third-party cookies**. Check out https://colab.research.google.com/notebooks/io.ipynb for other alternatives.

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving gc-creds.json to gc-creds.json
User uploaded file "gc-creds.json" with length 2314 bytes


In [0]:
!pwd
!ls -l ./gc-creds.json

/content
-rw-r--r-- 1 root root 2314 Jan 30 00:20 ./gc-creds.json


2. **Set environment variable**

In [0]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/gc-creds.json'

!ls -l $GOOGLE_APPLICATION_CREDENTIALS

-rw-r--r-- 1 root root 2314 Jan 30 00:20 /content/gc-creds.json


## Batch API

In [0]:
from google.cloud import speech_v1
from google.cloud.speech_v1 import enums

def google_batch_stt(filename: str, lang: str, encoding: str) -> str:
    buffer, rate = read_wav_file(filename)
    client = speech_v1.SpeechClient()

    config = {
        'language_code': lang,
        'sample_rate_hertz': rate,
        'encoding': enums.RecognitionConfig.AudioEncoding[encoding]
    }

    audio = {
        'content': buffer
    }

    response = client.recognize(config, audio)
    # For bigger audio file, the previous line can be replaced with following:
    # operation = client.long_running_recognize(config, audio)
    # response = operation.result()

    for result in response.results:
        # First alternative is the most probable result
        alternative = result.alternatives[0]
        return alternative.transcript

# Run tests
for t in TESTCASES:
    print('\naudio file="{0}"    expected text="{1}"'.format(t['filename'], t['text']))
    print('google-cloud-batch-stt: "{}"'.format(
        google_batch_stt(t['filename'], t['lang'], t['encoding'])
    ))


audio file="audio/2830-3980-0043.wav"    expected text="experience proves this"
google-cloud-batch-stt: "experience proves this"

audio file="audio/4507-16021-0012.wav"    expected text="why should one halt on the way"
google-cloud-batch-stt: "why should one halt on the way"

audio file="audio/8455-210777-0068.wav"    expected text="your power is sufficient i said"
google-cloud-batch-stt: "your power is sufficient I said"


## Streaming API

In [0]:
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types

def response_stream_processor(responses):
    print('interim results: ')

    transcript = ''
    num_chars_printed = 0
    for response in responses:
        if not response.results:
            continue

        result = response.results[0]
        if not result.alternatives:
            continue

        transcript = result.alternatives[0].transcript
        print('{0}final: {1}'.format(
            '' if result.is_final else 'not ',
            transcript
        ))

    return transcript

def google_streaming_stt(filename: str, lang: str, encoding: str) -> str:
    buffer, rate = read_wav_file(filename)

    client = speech.SpeechClient()

    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding[encoding],
        sample_rate_hertz=rate,
        language_code=lang
    )

    streaming_config = types.StreamingRecognitionConfig(
        config=config,
        interim_results=True
    )

    audio_generator = simulate_stream(buffer)  # buffer chunk generator
    requests = (types.StreamingRecognizeRequest(audio_content=chunk) for chunk in audio_generator)
    responses = client.streaming_recognize(streaming_config, requests)
    # Now, put the transcription responses to use.
    return response_stream_processor(responses)

# Run tests
for t in TESTCASES:
    print('\naudio file="{0}"    expected text="{1}"'.format(t['filename'], t['text']))
    print('google-cloud-streaming-stt: "{}"'.format(
        google_streaming_stt(t['filename'], t['lang'], t['encoding'])
    ))


audio file="audio/2830-3980-0043.wav"    expected text="experience proves this"
interim results: 
not final: next
not final: iSpy
not final: Aspira
not final: Xperia
not final: Experian
not final: experience
not final: experience proved
not final: experience proves
not final: experience proves the
not final: experience proves that
not final: experience
final: experience proves this
google-cloud-streaming-stt: "experience proves this"

audio file="audio/4507-16021-0012.wav"    expected text="why should one halt on the way"
interim results: 
not final: what
not final: watch
not final: why should
not final: why should we
not final: why should one
not final: why should one who
not final: why should one have
not final: why should
not final: why should
not final: why should
not final: why should
not final: why should one
not final: why should one
not final: why should one
not final: why should one
not final: why should one halt
not final: why should one halt on
not final: why should one hal


---

# Microsoft Azure

Microsoft Azure [Speech Services](https://azure.microsoft.com/en-in/services/cognitive-services/speech-services/) have [Speech to Text](https://azure.microsoft.com/en-in/services/cognitive-services/speech-to-text/) service.

## Setup

1. **Install azure speech package**

In [0]:
!pip3 install azure-cognitiveservices-speech

     |████████████████████████████████| 3.9MB 3.4MB/s 


2. **Set service credentials**

You can enable Speech service and find credentials for your account at [Microsoft Azure portal](https://portal.azure.com/). You can open a free account [here](https://azure.microsoft.com/en-in/free/ai/).

In [0]:
AZURE_SPEECH_KEY = 'YOUR AZURE SPEECH KEY'
AZURE_SERVICE_REGION = 'YOUR AZURE SERVICE REGION'

## Batch API

In [0]:
import azure.cognitiveservices.speech as speechsdk

def azure_batch_stt(filename: str, lang: str, encoding: str) -> str:
    speech_config = speechsdk.SpeechConfig(
        subscription=AZURE_SPEECH_KEY,
        region=AZURE_SERVICE_REGION
    )
    audio_input = speechsdk.AudioConfig(filename=filename)
    speech_recognizer = speechsdk.SpeechRecognizer(
        speech_config=speech_config,
        audio_config=audio_input
    )
    result = speech_recognizer.recognize_once()

    return result.text if result.reason == speechsdk.ResultReason.RecognizedSpeech else None

# Run tests
for t in TESTCASES:
    print('\naudio file="{0}"    expected text="{1}"'.format(t['filename'], t['text']))
    print('azure-batch-stt: "{}"'.format(
        azure_batch_stt(t['filename'], t['lang'], t['encoding'])
    ))


audio file="audio/2830-3980-0043.wav"    expected text="experience proves this"
azure-batch-stt: "Experience proves this."

audio file="audio/4507-16021-0012.wav"    expected text="why should one halt on the way"
azure-batch-stt: "Whi should one halt on the way."

audio file="audio/8455-210777-0068.wav"    expected text="your power is sufficient i said"
azure-batch-stt: "Your power is sufficient I said."


## Streaming API

In [0]:
import time
import azure.cognitiveservices.speech as speechsdk

def azure_streaming_stt(filename: str, lang: str, encoding: str) -> str:
    speech_config = speechsdk.SpeechConfig(
        subscription=AZURE_SPEECH_KEY,
        region=AZURE_SERVICE_REGION
    )
    stream = speechsdk.audio.PushAudioInputStream()
    audio_config = speechsdk.audio.AudioConfig(stream=stream)
    speech_recognizer = speechsdk.SpeechRecognizer(
        speech_config=speech_config,
        audio_config=audio_config
    )

    # Connect callbacks to the events fired by the speech recognizer
    speech_recognizer.recognizing.connect(lambda evt: print('interim text: "{}"'.format(evt.result.text)))
    speech_recognizer.recognized.connect(lambda evt:  print('azure-streaming-stt: "{}"'.format(evt.result.text)))

    # start continuous speech recognition
    speech_recognizer.start_continuous_recognition()

    # push buffer chunks to stream
    buffer, rate = read_wav_file(filename)
    audio_generator = simulate_stream(buffer)
    for chunk in audio_generator:
      stream.write(chunk)
      time.sleep(0.1)  # to give callback a chance against this fast loop

    # stop continuous speech recognition
    stream.close()
    time.sleep(0.5)  # give chance to VAD to kick in
    speech_recognizer.stop_continuous_recognition()
    time.sleep(0.5)  # Let all callback run

# Run tests
for t in TESTCASES:
    print('\naudio file="{0}"    expected text="{1}"'.format(t['filename'], t['text']))
    azure_streaming_stt(t['filename'], t['lang'], t['encoding'])


audio file="audio/2830-3980-0043.wav"    expected text="experience proves this"
interim text: "experience"
interim text: "experienced"
interim text: "experience"
interim text: "experience proves"
interim text: "experience proves this"
azure-streaming-stt: "Experience proves this."

audio file="audio/4507-16021-0012.wav"    expected text="why should one halt on the way"
interim text: "huaisheng"
interim text: "white"
interim text: "whi should"
interim text: "whi should one"
interim text: "whi should one halt"
interim text: "whi should one halt on"
interim text: "whi should one halt on the"
interim text: "whi should one halt on the way"
azure-streaming-stt: "Whi should one halt on the way."

audio file="audio/8455-210777-0068.wav"    expected text="your power is sufficient i said"
interim text: "you're"
interim text: "your"
interim text: "your power"
interim text: "your"
interim text: "your power is"
interim text: "your power is sufficient"
interim text: "your power is sufficient i"
int

---

# IBM Watson

For IBM [Watson Speech to Text](https://www.ibm.com/in-en/cloud/watson-speech-to-text) is ASR service with .NET, Go, JavaScript, [Python](https://cloud.ibm.com/apidocs/speech-to-text/speech-to-text?code=python), Ruby, Swift and Unity API libraries, as well as REST endpoints.


## Setup

1. **Install IBM Watson package**

In [0]:
!pip install ibm-watson

     |████████████████████████████████| 358kB 3.4MB/s 
     |████████████████████████████████| 204kB 43.6MB/s 
  Created wheel for ibm-watson: filename=ibm_watson-4.2.1-cp36-none-any.whl size=343298 sha256=3fcdea1185ceb522ed5f080ad4d66048d9286cd28e8d9bc86094b08a84cb6211
  Stored in directory: /root/.cache/pip/wheels/ce/4d/6e/ae352b7c7acdddf073aeb06617fbfeefaea9fcb6d7ae98800b
  Created wheel for ibm-cloud-sdk-core: filename=ibm_cloud_sdk_core-1.5.1-cp36-none-any.whl size=44492 sha256=8fbd5fdfa4ca15217877ee44671387c23ce61f390cd15d8006200d502d56dc63
  Stored in directory: /root/.cache/pip/wheels/6a/42/50/f96888116b329578304f9dda4693cef6f3e76e18272d22cb6c
Successfully built ibm-watson ibm-cloud-sdk-core


2. **Set service credentials**

You will need to [sign up/in](https://cloud.ibm.com/docs/services/text-to-speech?topic=text-to-speech-gettingStarted), and get API key credential and service URL, and fill it below.

In [0]:
WATSON_API_KEY = 'YOUR WATSON API KEY'
WATSON_STT_URL = 'YOUR WATSON SERVICE URL'

## Batch API

In [0]:
import os

from ibm_watson import SpeechToTextV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

def watson_batch_stt(filename: str, lang: str, encoding: str) -> str:
    authenticator = IAMAuthenticator(WATSON_API_KEY)
    speech_to_text = SpeechToTextV1(authenticator=authenticator)
    speech_to_text.set_service_url(WATSON_STT_URL)

    with open(filename, 'rb') as audio_file:
        response = speech_to_text.recognize(
            audio=audio_file,
            content_type='audio/{}'.format(os.path.splitext(filename)[1][1:]),
            model=lang + '_BroadbandModel',
            max_alternatives=3,
        ).get_result()

    return response['results'][0]['alternatives'][0]['transcript']

# Run tests
for t in TESTCASES:
    print('\naudio file="{0}"    expected text="{1}"'.format(t['filename'], t['text']))
    print('watson-batch-stt: "{}"'.format(
        watson_batch_stt(t['filename'], t['lang'], t['encoding'])
    ))


audio file="audio/2830-3980-0043.wav"    expected text="experience proves this"
watson-batch-stt: "experience proves this "

audio file="audio/4507-16021-0012.wav"    expected text="why should one halt on the way"
watson-batch-stt: "why should one hold on the way "

audio file="audio/8455-210777-0068.wav"    expected text="your power is sufficient i said"
watson-batch-stt: "your power is sufficient I set "


## Streaming API

Streaming API works over websocket.

In [0]:
import json
import logging
import os
from queue import Queue
from threading import Thread
import time

from ibm_watson import SpeechToTextV1
from ibm_watson.websocket import RecognizeCallback, AudioSource
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

# Watson websocket prints justs too many debug logs, so disable it
logging.disable(logging.CRITICAL)

# Chunk and buffer size
CHUNK_SIZE = 4096
BUFFER_MAX_ELEMENT = 10

# A callback class to process various streaming STT events
class MyRecognizeCallback(RecognizeCallback):
    def __init__(self):
        RecognizeCallback.__init__(self)
        self.transcript = None

    def on_transcription(self, transcript):
        # print('transcript: {}'.format(transcript))
        pass

    def on_connected(self):
        # print('Connection was successful')
        pass

    def on_error(self, error):
        # print('Error received: {}'.format(error))
        pass

    def on_inactivity_timeout(self, error):
        # print('Inactivity timeout: {}'.format(error))
        pass

    def on_listening(self):
        # print('Service is listening')
        pass

    def on_hypothesis(self, hypothesis):
        # print('hypothesis: {}'.format(hypothesis))
        pass

    def on_data(self, data):
        self.transcript = data['results'][0]['alternatives'][0]['transcript']
        print('{0}final: {1}'.format(
            '' if data['results'][0]['final'] else 'not ',
            self.transcript
        ))

    def on_close(self):
        # print("Connection closed")
        pass

def watson_streaming_stt(filename: str, lang: str, encoding: str) -> str:
    authenticator = IAMAuthenticator(WATSON_API_KEY)
    speech_to_text = SpeechToTextV1(authenticator=authenticator)
    speech_to_text.set_service_url(WATSON_STT_URL)

    # Make watson audio source fed by a buffer queue
    buffer_queue = Queue(maxsize=BUFFER_MAX_ELEMENT)
    audio_source = AudioSource(buffer_queue, True, True)

    # Callback object
    mycallback = MyRecognizeCallback()

    # Read the file
    buffer, rate = read_wav_file(filename)

    # Start Speech-to-Text recognition thread
    stt_stream_thread = Thread(
        target=speech_to_text.recognize_using_websocket,
        kwargs={
            'audio': audio_source,
            'content_type': 'audio/l16; rate={}'.format(rate),
            'recognize_callback': mycallback,
            'interim_results': True
        }
    )
    stt_stream_thread.start()

    # Simulation audio stream by breaking file into chunks and filling buffer queue
    audio_generator = simulate_stream(buffer, CHUNK_SIZE)
    for chunk in audio_generator:
        buffer_queue.put(chunk)
        time.sleep(0.5)  # give a chance to callback

    # Close the audio feed and wait for STTT thread to complete
    audio_source.completed_recording()
    stt_stream_thread.join()

    # send final result
    return mycallback.transcript

# Run tests
for t in TESTCASES:
    print('\naudio file="{0}"    expected text="{1}"'.format(t['filename'], t['text']))
    print('watson-cloud-streaming-stt: "{}"'.format(
        watson_streaming_stt(t['filename'], t['lang'], t['encoding'])
    ))


audio file="audio/2830-3980-0043.wav"    expected text="experience proves this"
not final: X. 
not final: experts 
not final: experience 
not final: experienced 
not final: experience prove 
not final: experience proves 
not final: experience proves that 
not final: experience proves this 
final: experience proves this 
watson-cloud-streaming-stt: "experience proves this "

audio file="audio/4507-16021-0012.wav"    expected text="why should one halt on the way"
not final: why 
not final: what 
not final: why should 
not final: why should we 
not final: why should one 
not final: why should one whole 
not final: why should one hold 
not final: why should one hold on 
not final: why should one hold on the 
not final: why should one hold on the way 
final: why should one hold on the way 
watson-cloud-streaming-stt: "why should one hold on the way "

audio file="audio/8455-210777-0068.wav"    expected text="your power is sufficient i said"
not final: your 
not final: your power 
not final


---

# CMU Sphinx

[CMUSphinx](https://cmusphinx.github.io/) is has been around for quite some time, and has been adapting to advancements in ASR technologies. [PocketSphinx](https://github.com/cmusphinx/pocketsphinx-python) is speech-to-text decoder software package.

## Setup

1. **Install swig**

For macOS:

In [0]:
!brew install swig
!swig -version

For Linux:

In [0]:
!apt-get install -y swig libpulse-dev
!swig -version

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libpulse-mainloop-glib0 swig3.0
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  libpulse-dev libpulse-mainloop-glib0 swig swig3.0
0 upgraded, 4 newly installed, 0 to remove and 7 not upgraded.
Need to get 1,204 kB of archives.
After this operation, 6,538 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libpulse-mainloop-glib0 amd64 1:11.1-1ubuntu7.4 [22.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libpulse-dev amd64 1:11.1-1ubuntu7.4 [81.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Get:4 http://

2. **Install poocketsphinx using pip**

In [0]:
!pip3 install pocketsphinx
!pip3 list | grep pocketsphinx

     |████████████████████████████████| 29.1MB 102kB/s 
  Created wheel for pocketsphinx: filename=pocketsphinx-0.1.15-cp36-cp36m-linux_x86_64.whl size=30126870 sha256=d111bc1a768251e9b8b4bea71f05b498955eda209f5d5650f7e68cc336bb5075
  Stored in directory: /root/.cache/pip/wheels/52/fd/52/2f62c9a0036940cc0c89e58ee0b9d00fcf78243aeaf416265f
Successfully built pocketsphinx
pocketsphinx                   0.1.15     


## Create Decoder object

In [0]:
import pocketsphinx
import os

MODELDIR = os.path.join(os.path.dirname(pocketsphinx.__file__), 'model')

config = pocketsphinx.Decoder.default_config()
config.set_string('-hmm', os.path.join(MODELDIR, 'en-us'))
config.set_string('-lm', os.path.join(MODELDIR, 'en-us.lm.bin'))
config.set_string('-dict', os.path.join(MODELDIR, 'cmudict-en-us.dict'))

decoder = pocketsphinx.Decoder(config)

## Batch API

In [0]:
def sphinx_batch_stt(filename: str, lang: str, encoding: str) -> str:
    buffer, rate = read_wav_file(filename)
    decoder.start_utt()
    decoder.process_raw(buffer, False, False)
    decoder.end_utt()
    hypothesis = decoder.hyp()
    return hypothesis.hypstr

# Run tests
for t in TESTCASES:
    print('\naudio file="{0}"    expected text="{1}"'.format(t['filename'], t['text']))
    print('sphinx-batch-stt: "{}"'.format(
        sphinx_batch_stt(t['filename'], t['lang'], t['encoding'])
    ))


audio file="audio/2830-3980-0043.wav"    expected text="experience proves this"
sphinx-batch-stt: "experience proves this"

audio file="audio/4507-16021-0012.wav"    expected text="why should one halt on the way"
sphinx-batch-stt: "why should one hold on the way"

audio file="audio/8455-210777-0068.wav"    expected text="your power is sufficient i said"
sphinx-batch-stt: "your paris sufficient i said"


## Streaming API

In [0]:
def sphinx_streaming_stt(filename: str, lang: str, encoding: str) -> str:
    buffer, rate = read_wav_file(filename)
    audio_generator = simulate_stream(buffer)

    decoder.start_utt()
    for chunk in audio_generator:
        decoder.process_raw(chunk, False, False)
    decoder.end_utt()

    hypothesis = decoder.hyp()
    return hypothesis.hypstr

# Run tests
for t in TESTCASES:
    print('\naudio file="{0}"    expected text="{1}"'.format(t['filename'], t['text']))
    print('sphinx-streaming-stt: "{}"'.format(
        sphinx_streaming_stt(t['filename'], t['lang'], t['encoding'])
    ))


audio file="audio/2830-3980-0043.wav"    expected text="experience proves this"
sphinx-streaming-stt: "experience proves this"

audio file="audio/4507-16021-0012.wav"    expected text="why should one halt on the way"
sphinx-streaming-stt: "why should one hold on the way"

audio file="audio/8455-210777-0068.wav"    expected text="your power is sufficient i said"
sphinx-streaming-stt: "your paris sufficient i said"



---

# Mozilla DeepSpeech

Mozilla released [DeepSpeech 0.6](https://hacks.mozilla.org/2019/12/deepspeech-0-6-mozillas-speech-to-text-engine/) software package in December 2019 with [APIs](https://github.com/mozilla/DeepSpeech/releases/tag/v0.6.0) in C, Java, .NET, [Python](https://deepspeech.readthedocs.io/en/v0.6.0/Python-API.html), and JavaScript, including support for TensorFlow Lite models for use on edge devices.

## Setup

1. **Install DeepSpeech**

You can install DeepSpeech with pip (make it deepspeech-gpu==0.6.0 if you are using GPU in colab runtime).

In [0]:
!pip install deepspeech==0.6.0

     |████████████████████████████████| 9.6MB 3.0MB/s 


2. **Download and unzip models**

In [0]:
!curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.6.0/deepspeech-0.6.0-models.tar.gz
!tar -xvzf deepspeech-0.6.0-models.tar.gz
!ls -l ./deepspeech-0.6.0-models/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   620    0   620    0     0   2857      0 --:--:-- --:--:-- --:--:--  2857
100 1172M  100 1172M    0     0  48.9M      0  0:00:23  0:00:23 --:--:-- 56.8M
deepspeech-0.6.0-models/
deepspeech-0.6.0-models/lm.binary
deepspeech-0.6.0-models/output_graph.pbmm
deepspeech-0.6.0-models/output_graph.pb
deepspeech-0.6.0-models/trie
deepspeech-0.6.0-models/output_graph.tflite
total 1350664
-rw-r--r-- 1 501 staff 945699324 Dec  3 06:51 lm.binary
-rw-r--r-- 1 501 staff 188914896 Dec  3 09:03 output_graph.pb
-rw-r--r-- 1 501 staff 188915850 Dec  3 09:49 output_graph.pbmm
-rw-r--r-- 1 501 staff  47335752 Dec  3 09:05 output_graph.tflite
-rw-r--r-- 1 501 staff  12200736 Dec  3 06:51 trie


3. **Test that it all works**

Examine the output of the last three commands, and you will see results *“experience proof less”*, *“why should one halt on the way”*, and *“your power is sufficient i said”* respectively. You are all set.

In [0]:
!deepspeech --model deepspeech-0.6.0-models/output_graph.pb --lm deepspeech-0.6.0-models/lm.binary --trie ./deepspeech-0.6.0-models/trie --audio ./audio/2830-3980-0043.wav

Loading model from file deepspeech-0.6.0-models/output_graph.pb
TensorFlow: v1.14.0-21-ge77504a
DeepSpeech: v0.6.0-0-g6d43e21
2020-01-30 00:27:46.675441: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Loaded model in 0.13s.
Loading language model from files deepspeech-0.6.0-models/lm.binary ./deepspeech-0.6.0-models/trie
Loaded language model in 0.000221s.
Running inference.
experience proof less
Inference took 2.418s for 1.975s audio file.


In [0]:
!deepspeech --model deepspeech-0.6.0-models/output_graph.pb --lm deepspeech-0.6.0-models/lm.binary --trie ./deepspeech-0.6.0-models/trie --audio ./audio/4507-16021-0012.wav

Loading model from file deepspeech-0.6.0-models/output_graph.pb
TensorFlow: v1.14.0-21-ge77504a
DeepSpeech: v0.6.0-0-g6d43e21
2020-01-30 00:27:53.427469: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Loaded model in 0.131s.
Loading language model from files deepspeech-0.6.0-models/lm.binary ./deepspeech-0.6.0-models/trie
Loaded language model in 0.000188s.
Running inference.
why should one halt on the way
Inference took 2.941s for 2.735s audio file.


In [0]:
!deepspeech --model deepspeech-0.6.0-models/output_graph.pb --lm deepspeech-0.6.0-models/lm.binary --trie ./deepspeech-0.6.0-models/trie --audio ./audio/8455-210777-0068.wav

Loading model from file deepspeech-0.6.0-models/output_graph.pb
TensorFlow: v1.14.0-21-ge77504a
DeepSpeech: v0.6.0-0-g6d43e21
2020-01-30 00:28:00.365841: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Loaded model in 0.129s.
Loading language model from files deepspeech-0.6.0-models/lm.binary ./deepspeech-0.6.0-models/trie
Loaded language model in 0.000228s.
Running inference.
your power is sufficient i said
Inference took 2.839s for 2.590s audio file.


## Create model object

In [0]:
import deepspeech

model_file_path = 'deepspeech-0.6.0-models/output_graph.pbmm'
beam_width = 500
model = deepspeech.Model(model_file_path, beam_width)

# Add language model for better accuracy
lm_file_path = 'deepspeech-0.6.0-models/lm.binary'
trie_file_path = 'deepspeech-0.6.0-models/trie'
lm_alpha = 0.75
lm_beta = 1.85
model.enableDecoderWithLM(lm_file_path, trie_file_path, lm_alpha, lm_beta)

0

## Batch API

In [0]:
import numpy as np

def deepspeech_batch_stt(filename: str, lang: str, encoding: str) -> str:
    buffer, rate = read_wav_file(filename)
    data16 = np.frombuffer(buffer, dtype=np.int16)
    return model.stt(data16)

# Run tests
for t in TESTCASES:
    print('\naudio file="{0}"    expected text="{1}"'.format(t['filename'], t['text']))
    print('deepspeech-batch-stt: "{}"'.format(
        deepspeech_batch_stt(t['filename'], t['lang'], t['encoding'])
    ))


audio file="audio/2830-3980-0043.wav"    expected text="experience proves this"
deepspeech-batch-stt: "experience proof less"

audio file="audio/4507-16021-0012.wav"    expected text="why should one halt on the way"
deepspeech-batch-stt: "why should one halt on the way"

audio file="audio/8455-210777-0068.wav"    expected text="your power is sufficient i said"
deepspeech-batch-stt: "your power is sufficient i said"


## Streaming API

In [0]:
def deepspeech_streaming_stt(filename: str, lang: str, encoding: str) -> str:
    buffer, rate = read_wav_file(filename)
    audio_generator = simulate_stream(buffer)

    # Create stream
    context = model.createStream()

    text = ''
    for chunk in audio_generator:
        data16 = np.frombuffer(chunk, dtype=np.int16)
        # feed stream of chunks
        model.feedAudioContent(context, data16)
        interim_text = model.intermediateDecode(context)
        if interim_text != text:
            text = interim_text
            print('inetrim text: {}'.format(text))

    # get final resut and close stream
    text = model.finishStream(context)
    return text

# Run tests
for t in TESTCASES:
    print('\naudio file="{0}"    expected text="{1}"'.format(t['filename'], t['text']))
    print('deepspeech-streaming-stt: "{}"'.format(
        deepspeech_streaming_stt(t['filename'], t['lang'], t['encoding'])
    ))


audio file="audio/2830-3980-0043.wav"    expected text="experience proves this"
inetrim text: i
inetrim text: e
inetrim text: experi en
inetrim text: experience pro
inetrim text: experience proof les
deepspeech-streaming-stt: "experience proof less"

audio file="audio/4507-16021-0012.wav"    expected text="why should one halt on the way"
inetrim text: i
inetrim text: why shou
inetrim text: why should one
inetrim text: why should one haul
inetrim text: why should one halt 
inetrim text: why should one halt on the 
deepspeech-streaming-stt: "why should one halt on the way"

audio file="audio/8455-210777-0068.wav"    expected text="your power is sufficient i said"
inetrim text: i
inetrim text: your p
inetrim text: your power is
inetrim text: your power is suffi
inetrim text: your power is sufficient i
inetrim text: your power is sufficient i said
deepspeech-streaming-stt: "your power is sufficient i said"



---

# SpeechRecognition Package

The [SpeechRecognition](https://pypi.org/project/SpeechRecognition/) package provide a nice abstraction over several solutions. In this notebook we explore using CMU Sphinx (i.e. model running locally on the machine), and Google (i.e. service accessed over the network/cloud), but both through SpeechRecognition package APIs.

## Setup

We need to install SpeechRecognition and pocketsphinx python packages, and download some files to test these APIs.

1. **Install SpeechRecognition py package**

In [0]:
!pip3 install SpeechRecognition

     |████████████████████████████████| 32.8MB 92kB/s 


Pocketsphinx has already been installed in earlier sections.

## Batch API

SpeechRecognition has only batch API. First step to create an audio record, eithher from a file or from mic, and the second step is to call `recognize_<speech engine name>` function. It currently has APIs for [CMU Sphinx, Google, Microsoft, IBM, Houndify, and Wit](https://github.com/Uberi/speech_recognition).

In [0]:
import speech_recognition as sr
from enum import Enum, unique

@unique
class ASREngine(Enum):
    sphinx = 0
    google = 1

def speech_to_text(filename: str, engine: ASREngine, language: str, show_all: bool = False) -> str:
    r = sr.Recognizer()

    with sr.AudioFile(filename) as source:
        audio = r.record(source)

    asr_functions = {
        ASREngine.sphinx: r.recognize_sphinx,
        ASREngine.google: r.recognize_google,
    }

    response = asr_functions[engine](audio, language=language, show_all=show_all)
    return response

# Run tests
for t in TESTCASES:
    filename = t['filename']
    text = t['text']
    lang = t['lang']

    print('\naudio file="{0}"    expected text="{1}"'.format(filename, text))
    for asr_engine in ASREngine:
        try:
            response = speech_to_text(filename, asr_engine, language=lang)
            print('{0}: "{1}"'.format(asr_engine.name, response))
        except sr.UnknownValueError:
            print('{0} could not understand audio'.format(asr_engine.name))
        except sr.RequestError as e:
            print('{0} error: {0}'.format(asr_engine.name, e))


audio file="audio/2830-3980-0043.wav"    expected text="experience proves this"
sphinx: "experience proves that"
google: "experience proves this"

audio file="audio/4507-16021-0012.wav"    expected text="why should one halt on the way"
sphinx: "why should one hold on the way"
google: "why should one halt on the way"

audio file="audio/8455-210777-0068.wav"    expected text="your power is sufficient i said"
sphinx: "your paris official said"
google: "your power is sufficient I said"


### API for other providers

For other speech recognition providers, you will need to create API credentials, which you have to pass to `recognize_<speech engine name>` function, you can checkout [this example](https://github.com/Uberi/speech_recognition/blob/master/examples/audio_transcribe.py).

It also has a nice abstraction for Microphone, implemented over PyAudio/PortAudio. Here is an example to capture input from mic in [batch](https://github.com/Uberi/speech_recognition/blob/master/examples/microphone_recognition.py) and continously in [background](https://github.com/Uberi/speech_recognition/blob/master/examples/background_listening.py).

---

# Summary

This note covers various available speech recognition:

- services: Google, Azure, Watson
- software: CMU Sphinx, Mozilla DeepSpeech

All of these have two kind of Speech-to-Text APIs:

- batch: the audio data is fed in one go
- streaming: the audio data is fed in chunks (very useful for transcribing microphone input)

The Python SpeechRecognition package provides abstraction over several speech recognition services and softwares.

I hope to include following in future:

- services: [Amazon Transcribe](https://aws.amazon.com/transcribe/), and [Nuance](https://nuancedev.github.io/samples/http/python/)
- software: [Kaldi](https://pykaldi.github.io/), and [Facebook wav2letter](https://ai.facebook.com/blog/online-speech-recognition-with-wav2letteranywhere/)

<br/>

---
&copy; 2020 Satish Chandra Gupta